<a href="https://colab.research.google.com/github/jinhqian/LING-L645/blob/main/Practical3_LM/Bigram_Perplexity/Bigram_perplexity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import sys, math, re, pickle
import nltk
from collections import defaultdict, Counter
from nltk.tokenize import word_tokenize
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [5]:
corpus_train = ["are you still here?",
"where are you?",
"are you tired?",
"i am tired.",
"are you in england?",
"were you in mexico?"]

corpus_test = ["where are you?",
"were you in england?",
"are you in mexico?",
"i am in mexico.",
"are you still in mexico?"]

In [6]:
count_bigrams, count_unigrams = defaultdict(Counter), Counter()
model = defaultdict(lambda : defaultdict(float)) 

def Ngrams(corpus):
    list_unigrams = []
    list_bigrams = []
    for line in corpus:
        tokens = word_tokenize(line)
        tokens = ['<BOS>'] + tokens
     
        for i in range(len(tokens)):
            count_unigrams[tokens[i]] += 1
            list_unigrams.append(tokens[i])
        
        for i in range(len(tokens) - 1):
            count_bigrams[tokens[i]][tokens[i+1]] += 1
            list_bigrams.append((tokens[i], tokens[i + 1]))
    return list_unigrams, list_bigrams 

list_unigrams,list_bigrams  = Ngrams(corpus_train)
print(f'Unigrams:\n{list_unigrams}', f'\ntotal unigrams: {len(list_unigrams)}')
print(f'\nBigrams:\n{list_bigrams}' f'\ntotal bigrams: {len(list_bigrams)}')
print(f'\nUnigram counts:\n{count_unigrams}')
print(f'\nBigram counts:\n{count_bigrams}')

# unigram probability
def unigrams_prob(unigrams, total_unigrams):
    unigrams_prob = {}
    for unigram in unigrams:
        unigrams_prob[unigram] = unigrams[unigram]/total_unigrams
    return unigrams_prob

# Probability of sentences (unigram)
def unigram_sentence_prob(unigram_prob): 
    unigram_sentence_prob = 1
    for p in unigram_prob.values():
        unigram_sentence_prob *= p
    return unigram_sentence_prob

unigram_prob = unigrams_prob(count_unigrams, len(list_unigrams))
print(f'\nUnigram probability:\n{unigram_prob}')
unigram_sentence_prob = unigram_sentence_prob(unigram_prob)
unigram_perplexity = unigram_sentence_prob ** (-1/len(list_unigrams)) # Unigram perplexity
print(f'\nUnigram perplexity:\n{unigram_perplexity}')

# bigram probability
for i in count_bigrams: 
    for j in count_bigrams[i]:
        model[i][j] = (count_bigrams[i][j] + 1) / (count_unigrams[i] + len(count_unigrams))
        # model[i][j] = count_bigrams[i][j] / count_unigrams[i]
print(f'\nBigram probability:\n{model}')
pickle.dump(dict(model), open('model.lm', 'wb'))

# Probability of sentences (bigram)
print('\nProbability of sentences (bigram):')
for line in corpus_train:
    tokens = word_tokenize(line)
    tokens = ['<BOS>'] + tokens
    # print(tokens)
    P = 1
    for i in range(len(tokens) - 1):  
        P *= model[tokens[i]][tokens[i+1]]
    
    print('%.6f' % P, '%.6f' % math.log(P), tokens)

# bigram perplexity
bigram_perplexity = P ** (-1/len(list_unigrams))
print(f'\nBigram perplexity:\n{bigram_perplexity}')
print('\n##### In train data #####')
print(f'Unigram perplexity: {unigram_perplexity}, Bigram perplexity: {bigram_perplexity}')
print("Bigram has lower perplexity than Unigram")
print('''
##### In test data #####
# Unigram perplexity: 3.3479024417796, Bigram perplexity: 1.4136346948208365
# Bigram has lower perplexity than Unigram ''')

Unigrams:
['<BOS>', 'are', 'you', 'still', 'here', '?', '<BOS>', 'where', 'are', 'you', '?', '<BOS>', 'are', 'you', 'tired', '?', '<BOS>', 'i', 'am', 'tired', '.', '<BOS>', 'are', 'you', 'in', 'england', '?', '<BOS>', 'were', 'you', 'in', 'mexico', '?'] 
total unigrams: 33

Bigrams:
[('<BOS>', 'are'), ('are', 'you'), ('you', 'still'), ('still', 'here'), ('here', '?'), ('<BOS>', 'where'), ('where', 'are'), ('are', 'you'), ('you', '?'), ('<BOS>', 'are'), ('are', 'you'), ('you', 'tired'), ('tired', '?'), ('<BOS>', 'i'), ('i', 'am'), ('am', 'tired'), ('tired', '.'), ('<BOS>', 'are'), ('are', 'you'), ('you', 'in'), ('in', 'england'), ('england', '?'), ('<BOS>', 'were'), ('were', 'you'), ('you', 'in'), ('in', 'mexico'), ('mexico', '?')]
total bigrams: 27

Unigram counts:
Counter({'<BOS>': 6, 'you': 5, '?': 5, 'are': 4, 'tired': 2, 'in': 2, 'still': 1, 'here': 1, 'where': 1, 'i': 1, 'am': 1, '.': 1, 'england': 1, 'were': 1, 'mexico': 1})

Bigram counts:
defaultdict(<class 'collections.Counter